In [7]:

!pip install pandas

     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ---------------------------------------- 60.9/60.9 kB ? eta 0:00:00
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---- ----------------------------------- 1.2/11.0 MB 26.7 MB/s eta 0:00:01
   --------------- ------------------------ 4.4/11.0 MB 55.8 MB/s eta 0:00:01
   --------------------------- ------------ 7.4/11.0 MB 53.0 MB/s eta 0:00:01
   ---------------------------------- ----- 9.6/11.0 MB 55.9 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 59.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   --------- ------------------------------ 3.0/12.8 MB 63.0 MB/s eta 0:00:01
   ----------------- ---------------------- 5.5/12.8 MB 70.8 MB/s eta 0:00:01
   --------------------------- ------------ 8.9/12.8 MB 63.1 MB/s eta 0:00:01
   ----------------------------------- ---- 11.3/12.8 MB 65.6 MB/s eta 0:00:01
   --------


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [54]:

import pandas as pd

# Load the CSV files
books = pd.read_csv('../Data/03_Library Systembook.csv', header=0, skiprows=0)
customers = pd.read_csv('../Data/03_Library SystemCustomers.csv', header=0, skiprows=0)


# Convert 'maximum_days_to_borrow' to number of days
def parse_borrow_period(period):
    if pd.isna(period):
        return None
    period = str(period).lower().strip()
    if 'week' in period:
        num = int(period.split()[0])
        return num * 7
    elif 'day' in period:
        num = int(period.split()[0])
        return num
    return None


books_ideal = (
    books
        .loc[books['Id'].notna()]
        .assign(
            book_pk=lambda d: d['Id'].astype('Int64'),
            book_name=lambda d: d['Books'],
            book_checkout_date=lambda d: pd.to_datetime(
                d['Book checkout'].str.replace('"', '').str.strip(), 
                format='%d/%m/%Y', 
                errors='coerce'
            ),
            book_returned_date=lambda d: pd.to_datetime(d['Book Returned'], format='%d/%m/%Y', errors='coerce'),
            maximum_days_to_borrow=lambda d: d['Days allowed to borrow'], 
            book_customer_fk=lambda d: d['Customer ID'].astype('Int64'),
            
        )
        [['book_pk', 'book_name', 'book_checkout_date','book_returned_date','maximum_days_to_borrow','book_customer_fk']]
)


# Apply the parsing function
books_ideal['max_borrow_days'] = books_ideal['maximum_days_to_borrow'].apply(parse_borrow_period)

# Calculate the actual borrowing duration
books_ideal['borrow_duration'] = (books_ideal['book_returned_date'] - books_ideal['book_checkout_date']).dt.days

# Check if the book was returned late
books_ideal['returned_overdue'] = books_ideal['borrow_duration'] > books_ideal['max_borrow_days']


#print(books_ideal.head())

customers_ideal = (
    customers
        .loc[customers['Customer ID'].notna()]
        .assign(
            customer_pk=lambda d: d['Customer ID'].astype('Int64'),
            customer_name=lambda d: d['Customer Name'],
                    )
        [['customer_pk','customer_name']]
)


books_with_customers = (
    books_ideal
        .merge(customers_ideal, left_on='book_customer_fk', right_on='customer_pk', how='left')
        .drop(columns=['book_customer_fk']) 
)

# Preview the result
#print(books_with_customers.head())


# Group by Customer_pk and count the number of books borrowed

books_per_customer = (
    books_with_customers
        .groupby(['customer_pk', 'customer_name'])
        .agg(
            total_books_borrowed=('book_pk', 'count'),
            first_checkout_date=('book_checkout_date', 'min'),
            last_return_date=('book_returned_date', 'max')
        )
        .reset_index()
)

# Preview the result
print(books_per_customer.head())




# Display the first few rows
#print(customers_ideal.head())


   customer_pk   customer_name  total_books_borrowed first_checkout_date  \
0            1        Jane Doe                     3          2023-02-20   
1            2      John Smith                     5          2023-03-24   
2            3      Dan Reeves                     3          2023-03-29   
3            5  William Holden                     2          2023-04-02   
4            6   Jaztyn Forest                     1          2063-04-10   

  last_return_date  
0       2023-05-01  
1       2023-06-20  
2       2023-06-04  
3       2023-06-07  
4       2023-04-03  


In [56]:

!pip install sqlalchemy
!pip install sqlite3  


   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ------------------------------------ --- 1.9/2.1 MB 40.3 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 33.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/297.8 kB ? eta -:--:--
   --------------------------------------- 297.8/297.8 kB 19.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/43.9 kB ? eta -:--:--
   ---------------------------------------- 43.9/43.9 kB ? eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [57]:
from sqlalchemy import create_engine

# Create a SQLite database file named 'library.db' in the current directory
engine = create_engine('sqlite:///library.db')


books_with_customers.to_sql('books_with_customers', con=engine, if_exists='replace', index=False)

# Write books_per_customer to another table
books_per_customer.to_sql('books_per_customer', con=engine, if_exists='replace', index=False)


8

In [60]:

import pandas as pd
from sqlalchemy import create_engine

# Connect to the SQLite database
engine = create_engine('sqlite:///library.db')

# Example: Read the entire books_per_customer table
df = pd.read_sql('SELECT * FROM books_per_customer', con=engine)

# Example: Filter customers with more than 3 books
df_filtered = pd.read_sql('''
    SELECT * FROM books_per_customer
    WHERE total_books_borrowed > 2
''', con=engine)

print(df_filtered.head())


   customer_pk customer_name  total_books_borrowed  \
0            1      Jane Doe                     3   
1            2    John Smith                     5   
2            3    Dan Reeves                     3   

          first_checkout_date            last_return_date  
0  2023-02-20 00:00:00.000000  2023-05-01 00:00:00.000000  
1  2023-03-24 00:00:00.000000  2023-06-20 00:00:00.000000  
2  2023-03-29 00:00:00.000000  2023-06-04 00:00:00.000000  
